In [ ]:
from torch_geometric.datasets import Planetoid

# 加载Cora数据集
dataset = Planetoid(root='data/Cora', name='Cora')
data = dataset[0]  # 获取图数据

print("数据集信息:")
print(f"节点数: {data.num_nodes}")
print(f"边数: {data.num_edges}")
print(f"类别数: {dataset.num_classes}")
print(f"特征维度: {dataset.num_node_features}")


Processing...


数据集信息:
节点数: 2708
边数: 10556
类别数: 7
特征维度: 1433


Done!


定义GCN模型，与卷积网络不同，传参时还需要传入，edge_index，即邻接矩阵

In [3]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        # 第一层卷积 + ReLU
        x = F.relu(self.conv1(x, edge_index))
        # 第二层卷积 + LogSoftmax
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# 初始化模型
model = GCN(dataset.num_node_features, 16, dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


3、模型的训练和测试：

In [ ]:
import torch
import torch.nn.functional as F

# 定义训练函数
def train():
    # 将模型设置为训练模式，在训练模式下，某些层（如 Dropout）的行为会有所不同
    model.train()
    # 清空优化器中的梯度信息，避免梯度累积影响本次训练
    optimizer.zero_grad()
    # 前向传播：将节点特征 data.x 和边索引 data.edge_index 输入到模型中，得到模型的输出
    out = model(data.x, data.edge_index)
    # 计算损失：使用负对数似然损失函数（F.nll_loss）计算模型输出中训练节点的预测结果与真实标签之间的损失
    # data.train_mask 是一个布尔掩码，用于筛选出训练集中的节点
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    # 反向传播：计算损失函数关于模型参数的梯度
    loss.backward()
    # 优化器更新模型参数：根据计算得到的梯度，使用优化器（如 Adam）更新模型的参数
    optimizer.step()
    # 返回当前训练步骤的损失值
    return loss.item()

# 定义测试函数
def test():
    # 将模型设置为评估模式，在评估模式下，某些层（如 Dropout）的行为会与训练模式不同
    model.eval()
    # 前向传播：将节点特征 data.x 和边索引 data.edge_index 输入到模型中，得到模型的输出
    out = model(data.x, data.edge_index)
    # 预测类别：在模型输出的每个节点的类别概率分布中，取概率最大的类别作为预测类别
    pred = out.argmax(dim=1)
    # 初始化一个空列表，用于存储训练集、验证集和测试集的准确率
    accs = []
    # 遍历训练集、验证集和测试集的掩码
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        # 计算预测正确的节点数量：比较预测类别和真实类别，统计预测正确的节点数量
        correct = (pred[mask] == data.y[mask]).sum()
        # 计算准确率：预测正确的节点数量除以该集合中的节点总数
        acc = int(correct) / int(mask.sum())
        # 将准确率添加到列表中
        accs.append(acc)
    # 返回训练集、验证集和测试集的准确率
    return accs

# 训练过程
# 设定训练的总轮数为 200 轮
for epoch in range(200):
    # 调用训练函数进行一轮训练，并返回该轮的损失值
    loss = train()
    # 调用测试函数进行测试，得到训练集、验证集和测试集的准确率
    train_acc, val_acc, test_acc = test()
    # 每 10 轮打印一次训练信息，包括当前轮数、损失值、训练集准确率、验证集准确率和测试集准确率
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}")

Epoch: 0, Loss: 1.9515, Train Acc: 0.7714, Val Acc: 0.5040, Test Acc: 0.5570
Epoch: 10, Loss: 0.5511, Train Acc: 0.9857, Val Acc: 0.7900, Test Acc: 0.8030
Epoch: 20, Loss: 0.0878, Train Acc: 1.0000, Val Acc: 0.7780, Test Acc: 0.8020
Epoch: 30, Loss: 0.0226, Train Acc: 1.0000, Val Acc: 0.7740, Test Acc: 0.8070
Epoch: 40, Loss: 0.0128, Train Acc: 1.0000, Val Acc: 0.7720, Test Acc: 0.8020
Epoch: 50, Loss: 0.0121, Train Acc: 1.0000, Val Acc: 0.7780, Test Acc: 0.8060
Epoch: 60, Loss: 0.0138, Train Acc: 1.0000, Val Acc: 0.7840, Test Acc: 0.8090
Epoch: 70, Loss: 0.0157, Train Acc: 1.0000, Val Acc: 0.7800, Test Acc: 0.8080
Epoch: 80, Loss: 0.0164, Train Acc: 1.0000, Val Acc: 0.7760, Test Acc: 0.8100
Epoch: 90, Loss: 0.0161, Train Acc: 1.0000, Val Acc: 0.7740, Test Acc: 0.8120
Epoch: 100, Loss: 0.0153, Train Acc: 1.0000, Val Acc: 0.7760, Test Acc: 0.8090
Epoch: 110, Loss: 0.0144, Train Acc: 1.0000, Val Acc: 0.7820, Test Acc: 0.8090
Epoch: 120, Loss: 0.0136, Train Acc: 1.0000, Val Acc: 0.7800, T